In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
from pathlib import Path

# Get the parent directory (i.e. project root)
project_root = Path().resolve().parent.parent 
sys.path.insert(0, str(project_root))

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from functools import partial

from tqdm import tqdm

from src.embedding import CustomEmbedding
from src.transformer import EncoderDecoderTransformer
from src.utils import padding_collate_fn

from pre_training.text_summarization.dataset import TextSummarizationDataset